In [1]:
import pandas as pd
anime = pd.read_csv('anime_unknown_removed.csv')

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification",
                    model="j-hartmann/emotion-english-distilroberta-base",
                    top_k=None)
classifier("I love this!")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

C:\Users\abhis\PycharmProjects\anime-recommender\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhis\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528673090040684},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419779404997826},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.0016119909705594182},
  {'label': 'fear', 'score': 0.00041385178337804973}]]

In [8]:
import numpy as np
emotion_labels = {'anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral'}

MAL_ID = []

emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key = lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]['score'])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [10]:
for i in range(10):
    MAL_ID.append(anime['MAL_ID'][i])
    sentences = anime['tagged_syn'][i].split('.')
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [11]:
emotion_scores

{'anger': [np.float64(0.41939979791641235),
  np.float64(0.25198468565940857),
  np.float64(0.8421081304550171),
  np.float64(0.02479126863181591),
  np.float64(0.1093573346734047),
  np.float64(0.035262905061244965),
  np.float64(0.09254060685634613),
  np.float64(0.5398648977279663),
  np.float64(0.13219401240348816),
  np.float64(0.6850703358650208),
  np.float64(0.41939979791641235),
  np.float64(0.25198468565940857),
  np.float64(0.8421081304550171),
  np.float64(0.02479126863181591),
  np.float64(0.1093573346734047),
  np.float64(0.035262905061244965),
  np.float64(0.07298532873392105),
  np.float64(0.5398648977279663),
  np.float64(0.13219401240348816),
  np.float64(0.6850703358650208)],
 'disgust': [np.float64(0.20812442898750305),
  np.float64(0.41534900665283203),
  np.float64(0.3099406063556671),
  np.float64(0.022154100239276886),
  np.float64(0.267577201128006),
  np.float64(0.06384380906820297),
  np.float64(0.27159130573272705),
  np.float64(0.20800836384296417),
  np.fl

In [12]:
from tqdm import tqdm

emotion_labels = {'anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral'}
MAL_ID = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(anime))):
    MAL_ID.append(anime['MAL_ID'][i])
    sentences = anime['tagged_syn'][i].split('.')
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 11165/11165 [43:40<00:00,  4.26it/s] 


In [13]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df['MAL_ID'] = MAL_ID

In [14]:
emotions_df.head()

,anger,disgust,fear,sadness,happiness,neutral,surprise,MAL_ID
0,0.419400,0.150204,0.649395,0.921934,0.930691,0.164440,0.118934,1
1,0.251985,0.415349,0.901187,0.040564,0.781068,0.111690,0.506986,5
2,0.842108,0.309941,0.395539,0.040564,0.908534,0.238724,0.078765,6
3,0.024791,0.022154,0.012809,0.013600,0.958140,0.519991,0.088976,7
4,0.109357,0.148752,0.509050,0.734795,0.906905,0.910966,0.078826,8


In [15]:
anime = pd.merge(anime, emotions_df, on='MAL_ID')

In [16]:
anime

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,sypnopsis,words_in_syn,tagged_syn,anger,disgust,fear,sadness,happiness,neutral,surprise
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,"In the year 2071, humanity has colonized sever...",169.0,"1 In the year 2071, humanity has colonized sev...",0.419400,0.150204,0.649395,0.921934,0.930691,0.164440,0.118934
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,"other day, another bounty—such is the life of ...",138.0,"5 other day, another bounty—such is the life o...",0.251985,0.415349,0.901187,0.040564,0.781068,0.111690,0.506986
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,"Vash the Stampede is the man with a $$60,000,0...",170.0,"6 Vash the Stampede is the man with a $$60,000...",0.842108,0.309941,0.395539,0.040564,0.908534,0.238724,0.078765
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,ches are individuals with special powers like ...,73.0,7 ches are individuals with special powers lik...,0.024791,0.022154,0.012809,0.013600,0.958140,0.519991,0.088976
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,It is the dark century and the people are suff...,130.0,8 It is the dark century and the people are su...,0.109357,0.148752,0.509050,0.734795,0.906905,0.910966,0.078826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11160,48466,Kyoukai Senki,Unknown,"Action, Mecha",Unknown,境界戦機,TV,Unknown,"Oct, 2021 to ?",Fall 2021,...,"In the year 2061 AD, Japan has lost its sovere...",89.0,"48466 In the year 2061 AD, Japan has lost its ...",0.391057,0.301095,0.321414,0.429227,0.935213,0.151205,0.140696
11161,48470,D_Cide Traumerei,Unknown,"Action, Adventure, Drama, Magic, Fantasy",Unknown,D_CIDE TRAUMEREI（ディーサイドトロイメライ）,TV,Unknown,"Jul, 2021 to ?",Summer 2021,...,The stage is Shibuya. When Ryuuhei Oda was in ...,79.0,48470 The stage is Shibuya. When Ryuuhei Oda w...,0.127037,0.524412,0.802501,0.936457,0.742492,0.032892,0.236868
11162,48471,Tsuki to Laika to Nosferatu,Unknown,"Sci-Fi, Space, Vampire",Unknown,月とライカと吸血姫,TV,Unknown,2021 to ?,Unknown,...,The first astronaut in human history was a vam...,152.0,48471 The first astronaut in human history was...,0.374029,0.415981,0.190098,0.407181,0.935213,0.095135,0.121303
11163,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",Unknown,見える子ちゃん,TV,Unknown,2021 to ?,Unknown,...,ko is a typical high school student whose life...,95.0,48483 ko is a typical high school student whos...,0.064134,0.634417,0.986944,0.040564,0.549477,0.111690,0.078765


In [17]:
anime.to_csv('anime_with_emotions.csv', index=False)